In [1]:
!pip install -qU langgraph langchain_core langchain-openai
!pip install -qU pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.9/469.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.4 MB/s eta 0:00:

In [4]:
from typing import Annotated, List
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph.message import MessagesState
from langchain_core.messages.modifier import RemoveMessage
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
import random

In [10]:
from typing import TypedDict

# Define annotated state for messages
class ChatState(TypedDict):
    # Modern LangGraph (2025): reducers are defined via annotation
    messages: Annotated[List[BaseMessage], add_messages]

In [6]:
# Example reducer that appends new messages to existing history
def append_reducer(existing: List, new: List):
    return existing + new

# Example reducer that trims the message history to last N messages
def trim_reducer(existing: List, new: List, max_messages=5):
    combined = existing + new
    if len(combined) > max_messages:
        combined = combined[-max_messages:]
    return combined

In [7]:
def user_node(state: ChatState):
    """Simulate user sending a message."""
    msg = HumanMessage(content=random.choice([
        "Hello!", "How are you?", "Tell me something cool.", "Let's talk about AI."
    ]))
    return {"messages": [msg]}

def ai_node(state: ChatState):
    """Simulate AI responding."""
    last_msg = state["messages"][-1].content if state["messages"] else "Hi!"
    response = AIMessage(content=f"AI Response to: {last_msg}")
    return {"messages": [response]}


In [12]:
# ✅ Cell 6 — Build Graph with Entry Point (2025 syntax)

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

# Initialize in-memory checkpointer
checkpointer = InMemorySaver()

# Create graph for our ChatState
graph = StateGraph(ChatState)

# Add nodes
graph.add_node("user", user_node)
graph.add_node("ai", ai_node)

# ✅ Define edges including entrypoint
graph.add_edge(START, "user")   # Entry point (new requirement)
graph.add_edge("user", "ai")
graph.add_edge("ai", END)

# ✅ Compile graph (reducers already inferred from Annotated state)
app = graph.compile(checkpointer=checkpointer)

print("✅ LangGraph compiled successfully (modern entrypoint & reducer syntax).")

✅ LangGraph compiled successfully (modern entrypoint & reducer syntax).


In [14]:
# ✅ Cell 7 — Run the Graph and View State

# Initial state — empty conversation
initial_state = {"messages": []}

# ✅ Each invocation must include a thread_id for the checkpointer
config = {"configurable": {"thread_id": "demo-thread"}}

# Run one full user → AI turn
final_state = app.invoke(initial_state, config=config)

# Display conversation history
print("🗨️ Conversation Messages:")
for msg in final_state["messages"]:
    print(f"{msg.type.upper()}: {msg.content}")

🗨️ Conversation Messages:
HUMAN: How are you?
AI: AI Response to: How are you?


In [15]:
# ✅ Cell 8 — Demonstrate Removing Messages (modern API)

from langchain_core.messages.modifier import RemoveMessage
from langchain_core.messages import BaseMessage

def remove_ai_messages_node(state: ChatState):
    """
    Node that removes all AI messages from the conversation.
    Demonstrates how RemoveMessage works with add_messages reducer.
    """
    msgs = state["messages"]
    # Collect RemoveMessage instructions for every AI message
    removals = [RemoveMessage(id=m.id) for m in msgs if m.type == "ai"]
    return {"messages": removals}


# Add the node to the graph for demonstration
graph = StateGraph(ChatState)
graph.add_node("user", user_node)
graph.add_node("ai", ai_node)
graph.add_node("remove_ai", remove_ai_messages_node)

# Define flow: START → user → ai → remove_ai → END
graph.add_edge(START, "user")
graph.add_edge("user", "ai")
graph.add_edge("ai", "remove_ai")
graph.add_edge("remove_ai", END)

# Compile with same checkpointer
app_remove = graph.compile(checkpointer=InMemorySaver())

# Run through the updated flow
config = {"configurable": {"thread_id": "remove-demo"}}
final_state = app_remove.invoke({"messages": []}, config=config)

print("🧽 After Removing AI Messages:")
for msg in final_state["messages"]:
    print(f"{msg.type.upper()}: {msg.content}")

🧽 After Removing AI Messages:
HUMAN: Tell me something cool.


In [16]:
# ✅ Cell 9 — Trimming & Summarizing Messages (2025 syntax)

from langchain_core.messages import AIMessage, HumanMessage

def summarize_old_messages(messages):
    """
    Simple summarization stub for demonstration.
    In production, you'd replace this with an LLM summarizer or LangChain Runnable.
    """
    # Concatenate content of all messages into a short summary
    combined_text = " | ".join(m.content for m in messages)
    return f"Summary: {combined_text[:120]}..."


def trim_and_summarize_node(state: ChatState, max_messages=4):
    """
    Trims long conversation histories and replaces removed parts with a summary.
    Works with the add_messages reducer.
    """
    msgs = state["messages"]

    # If conversation is short enough, no action needed
    if len(msgs) <= max_messages:
        return {"messages": []}

    # Messages to summarize (all but the last N)
    old_msgs = msgs[:-max_messages]
    recent_msgs = msgs[-max_messages:]

    # Create a new summary message
    summary = AIMessage(content=summarize_old_messages(old_msgs))

    # Return a replacement update:
    #  - Remove old messages
    #  - Add one summary message
    from langchain_core.messages.modifier import RemoveMessage
    removals = [RemoveMessage(id=m.id) for m in old_msgs]

    return {"messages": removals + [summary]}


# Build a small demo graph that includes the summarizer node
graph = StateGraph(ChatState)
graph.add_node("user", user_node)
graph.add_node("ai", ai_node)
graph.add_node("trim_summarize", trim_and_summarize_node)

# Flow: START → user → ai → trim_summarize → END
graph.add_edge(START, "user")
graph.add_edge("user", "ai")
graph.add_edge("ai", "trim_summarize")
graph.add_edge("trim_summarize", END)

# Compile the graph
app_trim = graph.compile(checkpointer=InMemorySaver())

# Run multiple turns to see trimming kick in
config = {"configurable": {"thread_id": "summary-demo"}}
state = {"messages": []}

for _ in range(6):  # simulate 6 turns
    state = app_trim.invoke(state, config=config)

print("🪶 After Trimming & Summarization:")
for msg in state["messages"]:
    print(f"{msg.type.upper()}: {msg.content}")

🪶 After Trimming & Summarization:
AI: AI Response to: How are you?
AI: Summary: AI Response to: Tell me something cool. | Summary: Hello! | AI Response to: Hello!... | Hello!...
HUMAN: Tell me something cool.
AI: AI Response to: Tell me something cool.
AI: Summary: AI Response to: Hello! | Summary: How are you? | AI Response to: How are you? | Tell me something cool.... | How are you...


In [17]:
# ✅ Cell 10 — Full Conversational Simulation (Modern LangGraph 2025)

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.messages.modifier import RemoveMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated, List
from langgraph.graph.message import add_messages

# --- State Definition ---
class ChatState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# --- Core Nodes ---
def user_node(state: ChatState):
    """Simulate user sending random messages."""
    import random
    msg = HumanMessage(content=random.choice([
        "Hey there!", "What's new in AI?", "Tell me something interesting.",
        "Can you summarize our chat?", "I'm learning LangGraph!"
    ]))
    return {"messages": [msg]}

def ai_node(state: ChatState):
    """AI echoes the latest user message."""
    last_msg = state["messages"][-1].content if state["messages"] else "Hi!"
    response = AIMessage(content=f"AI Response to: {last_msg}")
    return {"messages": [response]}

def remove_noise_node(state: ChatState):
    """
    Demonstrate removing specific messages.
    Example: remove AI messages that contain 'echo' or are too short.
    """
    removals = [
        RemoveMessage(id=m.id)
        for m in state["messages"]
        if m.type == "ai" and "echo" in m.content.lower()
    ]
    return {"messages": removals}

def summarize_old_messages(messages):
    """Simple summarization placeholder."""
    joined = " | ".join(m.content for m in messages)
    return f"Summary: {joined[:120]}..."

def trim_and_summarize_node(state: ChatState, max_messages=5):
    """Keep only last N messages, summarize the rest."""
    msgs = state["messages"]
    if len(msgs) <= max_messages:
        return {"messages": []}
    old, recent = msgs[:-max_messages], msgs[-max_messages:]
    summary = AIMessage(content=summarize_old_messages(old))
    removals = [RemoveMessage(id=m.id) for m in old]
    return {"messages": removals + [summary]}


# --- Graph Assembly ---
graph = StateGraph(ChatState)
graph.add_node("user", user_node)
graph.add_node("ai", ai_node)
graph.add_node("remove_noise", remove_noise_node)
graph.add_node("trim_summarize", trim_and_summarize_node)

# Flow: START → user → ai → remove_noise → trim_summarize → END
graph.add_edge(START, "user")
graph.add_edge("user", "ai")
graph.add_edge("ai", "remove_noise")
graph.add_edge("remove_noise", "trim_summarize")
graph.add_edge("trim_summarize", END)

# --- Compile ---
checkpointer = InMemorySaver()
app_full = graph.compile(checkpointer=checkpointer)

# --- Run Full Simulation ---
config = {"configurable": {"thread_id": "final-simulation"}}
state = {"messages": []}

print("🚀 Running full LangGraph chat simulation...\n")

for i in range(8):  # 8 turns to force trimming + summarization
    state = app_full.invoke(state, config=config)

print("\n💬 Final Conversation State:")
for msg in state["messages"]:
    print(f"{msg.type.upper()}: {msg.content}")

🚀 Running full LangGraph chat simulation...


💬 Final Conversation State:
HUMAN: Hey there!
AI: AI Response to: Hey there!
AI: Summary: Tell me something interesting. | AI Response to: Tell me something interesting. | Summary: Hey there! | AI Response to: ...
HUMAN: Can you summarize our chat?
AI: AI Response to: Can you summarize our chat?
AI: Summary: Can you summarize our chat? | AI Response to: Can you summarize our chat? | Summary: Can you summarize our chat? | AI Re...


In [19]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires l

In [21]:
# Install latest langchain-openai
!pip install -U langchain-openai

# Correct import
from langchain_openai import ChatOpenAI